<a href="https://colab.research.google.com/github/satoshikawato/gbdraw/blob/main/gbdraw_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gbdraw: a genome diagram generator for microbes

A command-line tool designed for creating detailed diagrams of microbial genomes. gbdraw accepts GenBank/EMBL/DDBJ-format annotated genomes as input and outputs a visual representation of the genomes in SVG/PNG/PDF/EPS/PS formats. This notebook allows you to try selected features of gbdraw on Google Colab without any local installation. For more details, see the [gbdraw GitHub](https://github.com/satoshikawato/gbdraw).

In [ ]:
#@title **1. Install gbdraw from GitHub repository**
%%time
import os

GBDRAW_READY = "GBDRAW_READY"

if not os.path.isfile(GBDRAW_READY):
    print("Cloning and installing gbdraw from GitHub...")
    if not os.path.isdir("gbdraw-main"):
        os.system("git clone https://github.com/satoshikawato/gbdraw.git gbdraw-main")

    os.chdir("gbdraw-main")
    os.system("pip install -q .")
    os.chdir("..")
    os.system(f"touch {GBDRAW_READY}")

print("Successfully installed")

## **2. Run gbdraw**

In [ ]:
#@title  Circular mode
import os, shutil, uuid
from IPython.display import display, Image, SVG, FileLink
from google.colab import files
import ipywidgets as wd

INPUT_DIR = "gbdraw_input"
OUT_DIR   = "gbdraw_output"
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUT_DIR,   exist_ok=True)

# ---------- helpers ----------
def _safe_name(fname: str) -> str:
    base = fname.replace(" ", "_").replace("(", "").replace(")", "")
    dst  = os.path.join(INPUT_DIR, base)
    if os.path.exists(dst):
        stem, ext = os.path.splitext(base)
        base = f"{stem}_{uuid.uuid4().hex[:6]}{ext}"
    return base

def _refresh_dropdowns(*_):
    """Scan INPUT_DIR and refresh file/options safely."""
    files = [""] + sorted(os.listdir(INPUT_DIR))      # allow blank selection
    gb_dd.options = files
    if gb_dd.value not in files:                      # keep widget valid
        gb_dd.value = ""
    track_dd.options = ["tuckin", "middle", "spreadout"]
    if track_dd.value not in track_dd.options:
        track_dd.value = "tuckin"
    fmt_select.options = ["png", "svg", "pdf", "eps", "ps", "png,svg"]
    if fmt_select.value not in fmt_select.options:
        fmt_select.value = "png"

# ---------- upload ----------
upload_btn = wd.Button(description="📁 Upload files")
def _uploader(_):
    up = files.upload()
    for raw in up:
        dst = _safe_name(raw)
        shutil.move(raw, os.path.join(INPUT_DIR, dst))
    _refresh_dropdowns()
    print("✅ Uploaded:", ", ".join(up.keys()))
upload_btn.on_click(_uploader)

# ---------- manual refresh ----------
refresh_btn = wd.Button(description="🔄 Refresh list")
refresh_btn.on_click(_refresh_dropdowns)

# ---------- widgets ----------
gb_dd       = wd.Dropdown(description="GenBank file:", options=[""])
track_dd    = wd.Dropdown(description="Track type:", options=["tuckin","middle","spreadout"], value="tuckin")
fmt_select  = wd.Dropdown(description="Output format:", options=["png","svg","pdf","eps","ps","png,svg"], value="png")
out_prefix  = wd.Text(value="circular", description="Output prefix:")

chk_labels  = wd.Checkbox(description="Show labels (--show_labels)")
chk_strands = wd.Checkbox(description="Separate strands (--separate_strands)")

# advanced
adv_nt   = wd.Text(value="GC", description="nt (--nt):")
adv_win  = wd.IntText(value=1000, description="window:")
adv_step = wd.IntText(value=100,  description="step:")
adv_feats= wd.Text(value="CDS,tRNA,rRNA,repeat_region", description="features (-k):")

adv_box = wd.VBox([
    wd.HTML("<b>Advanced options</b>"),
    wd.HBox([adv_nt, adv_win, adv_step]),
    adv_feats
])
adv_box.layout.display = "none"
toggle_adv = wd.ToggleButton(description="🔧 Advanced")
toggle_adv.observe(lambda c: setattr(adv_box.layout, "display", None if c["new"] else "none"), names="value")

# ---------- run ----------
run_btn = wd.Button(description="🚀 Run gbdraw", button_style="success")

def _run(_):
    if not gb_dd.value:
        print("❌ Please select a GenBank file.")
        return

    output_prefix = os.path.join(OUT_DIR, out_prefix.value)
    cmd = [
        "gbdraw", "circular",
        "-i", os.path.join(INPUT_DIR, gb_dd.value),
        "-o", output_prefix,
        "-f", fmt_select.value,
        "--track_type", track_dd.value
    ]
    if chk_labels.value:  cmd.append("--show_labels")
    if chk_strands.value: cmd.append("--separate_strands")
    if toggle_adv.value:
        cmd += ["-n", adv_nt.value,
                "-w", str(adv_win.value),
                "-s", str(adv_step.value),
                "-k", adv_feats.value]

    print("🛠️ Command:", " ".join(cmd))

    # -------- mtime-aware output detection --------
    before_mtime = {f: os.path.getmtime(os.path.join(OUT_DIR, f))
                    for f in os.listdir(OUT_DIR)}

    os.system(" ".join(cmd))                        # run gbdraw

    after_mtime = {f: os.path.getmtime(os.path.join(OUT_DIR, f))
                   for f in os.listdir(OUT_DIR)}

    changed = sorted(
        f for f, m in after_mtime.items()
        if f not in before_mtime or m > before_mtime[f] + 1e-6
    )
    if not changed:
        print("❌ No new or updated output files found.")
        return

    for f in changed:
        path = os.path.join(OUT_DIR, f)
        print("📌", f)
        if f.endswith(".svg"):
            display(SVG(filename=path))
        elif f.endswith(".png"):
            display(Image(filename=path))
        else:
            display(FileLink(path))

run_btn.on_click(_run)

# ---------- layout ----------
ui = wd.VBox([
    wd.HTML("<h2>🧬 gbdraw circular interface</h2>"),
    wd.HBox([upload_btn, refresh_btn]),
    gb_dd, track_dd, fmt_select, out_prefix,
    wd.HBox([chk_labels, chk_strands]),
    toggle_adv, adv_box,
    run_btn
])
_refresh_dropdowns()           # initial scan
display(ui)

In [ ]:
#@title  Linear mode
import os, shutil, uuid
from IPython.display import display, Image, SVG, FileLink
from google.colab import files
import ipywidgets as wd

INPUT_DIR = "gbdraw_input"      # shared with circular cell
OUT_DIR   = "gbdraw_output"
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUT_DIR,   exist_ok=True)

# ---------- helpers ----------
def _safe_name(fname: str) -> str:
    base = fname.replace(" ", "_").replace("(", "").replace(")", "")
    dst  = os.path.join(INPUT_DIR, base)
    if os.path.exists(dst):
        stem, ext = os.path.splitext(base)
        base = f"{stem}_{uuid.uuid4().hex[:6]}{ext}"
    return base

def _refresh_dropdowns(*_):
    files = [""] + sorted(os.listdir(INPUT_DIR))
    for d in SEQ_DDS + COMP_DDS:
        d.options = files
        if d.value not in files:
            d.value = ""
    # ensure fmt dropdown remains valid
    if fmt_dd.value not in fmt_dd.options:
        fmt_dd.value = "png"

def _rebuild_files_box():
    _refresh_dropdowns()
    children = []
    for i, seq_dd in enumerate(SEQ_DDS):
        children.append(seq_dd)
        if i < len(SEQ_DDS) - 1:
            children.append(COMP_DDS[i])
    FILE_BOX.children = tuple(children)

# ---------- upload ----------
upload_btn = wd.Button(description="📁 Upload files")
def _uploader(_):
    up = files.upload()
    for raw in up:
        dst = _safe_name(raw)
        shutil.move(raw, os.path.join(INPUT_DIR, dst))
    _refresh_dropdowns()
    print("✅ Uploaded:", ", ".join(up.keys()))
upload_btn.on_click(_uploader)

# ---------- manual refresh ----------
refresh_btn = wd.Button(description="🔄 Refresh list")
refresh_btn.on_click(_refresh_dropdowns)

# ---------- initial Sequence / Comparison widgets ----------
SEQ_DDS  = [wd.Dropdown(description="Sequence 1:", options=[""]),
            wd.Dropdown(description="Sequence 2:", options=[""])]
COMP_DDS = [wd.Dropdown(description="Comparison 1:", options=[""])]

more_btn = wd.Button(description="➕ Add pair")
def _add_pair(_):
    cid = len(COMP_DDS) + 1
    sid = len(SEQ_DDS)  + 1
    COMP_DDS.append(wd.Dropdown(description=f"Comparison {cid}:", options=[""]))
    SEQ_DDS.append(wd.Dropdown(description=f"Sequence {sid}:",   options=[""]))
    _rebuild_files_box()
more_btn.on_click(_add_pair)

# ---------- basic / advanced option widgets ----------
chk_align  = wd.Checkbox(description="Align center (--align_center)")
chk_sep    = wd.Checkbox(description="Separate strands (--separate_strands)")
chk_gc     = wd.Checkbox(description="Show GC (--show_gc)")
chk_labels = wd.Checkbox(description="Show labels (--show_labels)")
chk_resolv = wd.Checkbox(description="Resolve overlaps (--resolve_overlaps)")
legend_dd  = wd.Dropdown(description="Legend:", options=["none","left","right"], value="none")
fmt_dd     = wd.Dropdown(description="Output format:",
                         options=["png","svg","pdf","eps","ps","png,svg"],
                         value="png")
out_prefix = wd.Text(value="linear", description="Output prefix:")

adv_nt, adv_win, adv_step = (
    wd.Text(value="GC", description="nt (--nt):"),
    wd.IntText(value=1000, description="window:"),
    wd.IntText(value=100,  description="step:")
)
adv_bits  = wd.FloatText(value=50,   description="bitscore:")
adv_eval  = wd.FloatText(value=1e-2, description="evalue:")
adv_ident = wd.FloatText(value=0,    description="identity:")
adv_feats = wd.Text(value="CDS,tRNA,rRNA,repeat_region", description="features (-k):")

adv_box = wd.VBox([
    wd.HTML("<b>Advanced options</b>"),
    wd.HBox([adv_nt, adv_win, adv_step]),
    wd.HBox([adv_bits, adv_eval, adv_ident]),
    adv_feats
])
adv_box.layout.display = "none"
toggle_adv = wd.ToggleButton(description="🔧 Advanced")
toggle_adv.observe(lambda c: setattr(adv_box.layout, "display", None if c["new"] else "none"), names="value")

# ---------- run ----------
run_btn = wd.Button(description="🚀 Run gbdraw", button_style="success")

def _run(_):
    # --- validate Sequences ---
    seq_bases = [d.value for d in SEQ_DDS if d.value]
    if len(seq_bases) < 2:
        print("❌ Select at least two Sequence files.")
        return
    seq_files = [os.path.join(INPUT_DIR, f) for f in seq_bases]

    # --- validate Comparisons ---
    comp_bases = [d.value for d in COMP_DDS]
    filled     = [c for c in comp_bases if c]
    if filled and len(filled) != len(COMP_DDS):
        print("❌ Either fill all Comparison slots or leave all empty.")
        return
    comp_files = [os.path.join(INPUT_DIR, f) for f in filled]

    # --- assemble command ---
    cmd = [
        "gbdraw", "linear",
        "-i", *seq_files,
        "-o", os.path.join(OUT_DIR, out_prefix.value),
        "-f", fmt_dd.value,
        "-n", adv_nt.value, "-w", str(adv_win.value), "-s", str(adv_step.value),
        "--bitscore", str(adv_bits.value),
        "--evalue",  str(adv_eval.value),
        "--identity", str(adv_ident.value),
        "-k", adv_feats.value
    ]
    if chk_align.value:   cmd.append("--align_center")
    if chk_sep.value:     cmd.append("--separate_strands")
    if chk_gc.value:      cmd.append("--show_gc")
    if chk_labels.value:  cmd.append("--show_labels")
    if chk_resolv.value:  cmd.append("--resolve_overlaps")
    if legend_dd.value != "none":
        cmd += ["-l", legend_dd.value]
    if comp_files:
        cmd += ["-b", *comp_files]

    print("🛠️ Command:", " ".join(cmd))

    # --- mtime-aware output detection ---
    before_mtime = {f: os.path.getmtime(os.path.join(OUT_DIR, f))
                    for f in os.listdir(OUT_DIR)}

    os.system(" ".join(cmd))                 # run gbdraw

    after_mtime = {f: os.path.getmtime(os.path.join(OUT_DIR, f))
                   for f in os.listdir(OUT_DIR)}

    changed = sorted(
        f for f, m in after_mtime.items()
        if f not in before_mtime or m > before_mtime[f] + 1e-6
    )
    if not changed:
        print("❌ No new or updated output files found.")
        return

    for f in changed:
        p = os.path.join(OUT_DIR, f)
        print("📌", f)
        if f.endswith(".svg"):
            display(SVG(filename=p))
        elif f.endswith(".png"):
            display(Image(filename=p))
        else:
            display(FileLink(p))

run_btn.on_click(_run)

# ---------- layout ----------
FILE_BOX = wd.VBox()
_rebuild_files_box()

ui = wd.VBox([
    wd.HTML("<h2>🧬 gbdraw linear interface</h2>"),
    wd.HBox([upload_btn, refresh_btn]),
    FILE_BOX,
    more_btn,
    wd.HBox([chk_align, chk_sep]),
    wd.HBox([chk_gc, chk_labels]),
    wd.HBox([chk_resolv, legend_dd]),
    fmt_dd, out_prefix,
    toggle_adv, adv_box,
    run_btn
])
_refresh_dropdowns()      # initial scan
display(ui)
